# Vector 인덱스 만들기

Azure AI 검색에서 벡터 저장소에는 벡터 필드와 비 벡터 필드를 정의하는 인덱스 스키마, 포함 공간을 만들고 압축하는 알고리즘에 대한 벡터 구성, 쿼리 요청에 사용되는 벡터 필드 정의에 대한 설정이 있습니다.

인덱스 만들기 또는 업데이트 API는 벡터 저장소를 만듭니다. 다음 단계를 사용하여 벡터 데이터를 인덱싱합니다.

- 벡터 알고리즘과 선택적 압축을 사용하여 스키마 정의
- 벡터 필드 정의 추가
- 사전 벡터화된 데이터를 별도의 단계로 로드 또는 인덱싱 중에 통합 벡터화를 데이터 청크 및 인코딩에 사용

[필수 조건]

- 모든 지역 및 모든 계층의 Azure AI 검색. 대부분의 기존 서비스는 벡터 검색을 지원합니다. 2019년 1월 이전에 만들어진 서비스의 경우 벡터 인덱스를 만들 수 없는 작은 하위 집합이 있습니다. 이런 상황에서는 새로운 서비스를 만들어야 합니다. 통합 벡터화(Azure AI를 호출하는 기술 세트)를 사용하는 경우 Azure AI 검색은 Azure OpenAI 또는 Azure AI 서비스와 동일한 지역에 있어야 합니다.

- 기존 벡터 포함 또는 인덱싱 파이프라인에서 포함 모델을 호출하는 통합 벡터화를 사용합니다.

- 포함을 만드는 데 사용된 모델의 차원 제한을 알아야 합니다. 유효한 값은 2에서 3072까지입니다. Azure OpenAI에서 text-embedding-ada-002의 경우 숫자 벡터의 길이는 1536입니다. text-embedding-3-small 또는 text-embedding-3-large의 경우 벡터 길이는 3072입니다.

- 지원되는 유사성 메트릭이 무엇인지도 알아야 합니다. Azure OpenAI의 경우 유사성은 cosine를 사용하여 계산됩니다.

- 인덱스 만들기에 익숙해야 합니다. 스키마에는 문서 키에 대한 필드, 검색하거나 필터링하려는 기타 필드, 인덱싱 및 쿼리 중에 필요한 동작에 대한 기타 구성이 포함되어야 합니다.


> 📝 참고
>
> 더 자세한 내용은 [벡터 인덱스 만들기](https://learn.microsoft.com/ko-kr/azure/search/vector-search-how-to-create-index) 을 참고해 주세요.

```
작업 필요한 내용
- aoai 모델 배포 -> 참고 사이트 link 할것
```

In [28]:
import os
import json

from openai import AzureOpenAI

from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents import SearchIndexingBufferedSender
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.models import VectorizableTextQuery
from azure.search.documents.models import VectorFilterMode
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters
)

load_dotenv(override=True)

endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) if len(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) > 0 else DefaultAzureCredential()

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY", "") if len(os.getenv("AZURE_OPENAI_KEY", "")) > 0 else None
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-ada-002")
azure_openai_embedding_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 1536))
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01")

index_name = "vector_test"

### embedding 생성

In [2]:
client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

# text-embedding-ada-002로 embedding

# text-sample.json 로드
path = os.path.join('.', 'data', 'text-sample.json')
with open(path, 'r', encoding='utf-8') as file:
    input_data = json.load(file)

titles = [item['title'] for item in input_data]
content = [item['content'] for item in input_data]
title_response = client.embeddings.create(
    model = azure_openai_embedding_deployment,
    input = titles
)
title_embeddings = [item.embedding for item in title_response.data]
content_response = client.embeddings.create(
    model = azure_openai_embedding_deployment, 
    input = content
)
content_embeddings = [item.embedding for item in content_response.data]

# title, content 필드 임베딩
for i, item in enumerate(input_data):
    title = item['title']
    content = item['content']
    item['titleVector'] = title_embeddings[i]
    item['contentVector'] = content_embeddings[i]

# docVectors.json 파일로 임베딩 결과 기록
output_path = os.path.join('.', 'output', 'docVectors.json')
output_directory = os.path.dirname(output_path)
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
with open(output_path, "w") as f:
    json.dump(input_data, f)

### vector index 생성

In [8]:
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

# 필드 - titleVector, contentVector 필드가 vector 필드
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String, filterable=True),
    SearchField(name="titleVector",
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, 
                vector_search_dimensions=azure_openai_embedding_dimensions, 
                vector_search_profile_name="myHnswProfile"),
    SearchField(name="contentVector", 
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, 
                vector_search_dimensions=azure_openai_embedding_dimensions, 
                vector_search_profile_name="myHnswProfile"),
]

# 벡터 검색 구성  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer="myVectorizer"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            name = "myVectorizer",
            azure_open_ai_parameters = AzureOpenAIParameters(
                resource_uri = azure_openai_endpoint,
                deployment_id = azure_openai_embedding_deployment,
                model_name = azure_openai_embedding_deployment,
                api_key = azure_openai_key
            )
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="category")],
        content_fields=[SemanticField(field_name="content")]
    )
)

# 시맨틱 설정 
semantic_search = SemanticSearch(configurations=[semantic_config])

# 시맨틱 설정으로 index 생성
index = SearchIndex(
    name = index_name,
    fields = fields,
    vector_search = vector_search, 
    semantic_search = semantic_search)
result = index_client.create_or_update_index(index)

print(f' {result.name} created')

 vector_test created


### text와 embedding 된 vector 를 vector index로 입력

In [9]:
# output 디렉토리에서 embedding 된 결과물을 저장해 놓은 docVectors.json 를 읽어 indexing
 
output_path = os.path.join('.', 'output', 'docVectors.json')
output_directory = os.path.dirname(output_path)

if not os.path.exists(output_directory):
    os.makedirs(output_directory)
with open(output_path, 'r') as file:  
    documents = json.load(file)  

search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents") 

Uploaded 108 documents


In [ ]:
# 위의 셀을 실행 시켰다면, 이 셀은 실행 시키지 않아도 됩니다.
# 대량의 데이터를 indexing 할 경우, SearchIndexingBufferedSender를 사용하면 더욱 효율적으로  indexing 할 수 있습니다.

# with open(output_path, 'r') as file:  
#     documents = json.load(file)  
  
# with SearchIndexingBufferedSender(  
#     endpoint=endpoint,  
#     index_name=index_name,  
#     credential=credential,  
# ) as batch_client:  
#     # Add upload actions for all documents  
#     batch_client.upload_documents(documents=documents)  
# print(f"Uploaded {len(documents)} documents in total")  

### vector 유사도 검색

기본적인 벡터 검색 예시입니다.

[VectorizedQuery](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.models.vectorizedquery?view=azure-python) 참고

In [14]:
# vector 검색
query = "tools for software development"  

# query를 embedding 합니다.
embedding = client.embeddings.create(
    input = query,
    model = azure_openai_embedding_deployment
).data[0].embedding

# contentVector 필드에서 embdding 된 query로 검색하여 상위 3개의 가까운 항목 반환
vector_query = VectorizedQuery(
    vector = embedding, 
    k_nearest_neighbors = 3, 
    fields = "contentVector")
  
results = search_client.search(  
    search_text = None,  
    vector_queries = [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure DevOps
Score: 0.8289744
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.81774753
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such

### query를 embedding 하지 않고 검색 

텍스트 쿼리를 별도로 embedding 하지 않고, 텍스트 쿼리를 입력하면 vectorizer가 쿼리를 embedding 후 처리합니다.

[VectorizableTextQuery](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.models.vectorizabletextquery) 참고

In [16]:
query = "tools for software development"  
  
vector_query = VectorizableTextQuery(
    text=query, 
    k_nearest_neighbors=3, 
    fields="contentVector"
)
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure DevOps
Score: 0.8289744
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.81774753
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such

### Exhaustive KNN 검색

index에 설정 된 값이 아닌 Exhaustive KNN 으로 검색합니다. 정확하게 검색하기에 ground-truth  값으로 사용 할 수 있습니다.

In [17]:
query = "tools for software development"  

# 위의 예제와 다른 점은 exhaustive=True
vector_query = VectorizableTextQuery(
    text=query, 
    k_nearest_neighbors=3, 
    fields="contentVector", 
    exhaustive=True)
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure DevOps
Score: 0.8289743
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.8177473
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such 

### Cross-Field Vector 검색

여러 벡터 필드를 동시에 쿼리할 수 있는 교차 필드 벡터(Cross-Field Vector) 검색 입니다.</br>
contentVector, titleVector 벡터를 함께 검색합니다.

In [18]:
query = "tools for software development"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector, titleVector")

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure DevOps
Score: 0.03333333507180214
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.032786883413791656
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports var

### Multi-Vector 검색

- 여러 쿼리 벡터를 전달하여 여러 벡터 필드를 동시에 쿼리할 수 있는 교차 필드 벡터 검색을 보여줍니다.
- 이 경우 두 개의 서로 다른 임베딩 모델에서 인덱스의 해당 벡터 필드에 쿼리 벡터를 전달할 수 있습니다.
- 각각의 벡터 필드에 대해 벡터 검색을 수행하기, 가중치(weight), exhaustive KNN 등의 검색 설정도 다르게 부여 할 수 있습니다.

In [27]:
query = "tools for software development"  
  
vector_query_1 = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="titleVector", weight=1, exhaustive=True)
vector_query_2 = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector", weight=0.7)

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query_1, vector_query_2],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure DevOps
Score: 0.028333334252238274
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.027868852019309998
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports va

### Vector 검색에서 필터(filter) 사용

- 검색에 필터를 적용하는 방법을 보여줍니다.
- 사전 필터링(pre-filtering, 기본값)을 사용할지 사후 필터링(post-filtering)을 사용할지 선택할 수 있습니다.

In [29]:
query = "tools for software development"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector")

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="category eq 'Developer Tools'",
    select=["title", "content", "category"],
)
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure DevOps
Score: 0.8289743
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.8177473
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such 

### 하이브리드(Hybrid) 검색

어휘 검색과 벡터 검색을 함께 수행하여 결과를 반환 합니다.

In [30]:
query = "scalable storage solution"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector")

# 어휘 검색(search_text=query)과 벡터 검색(vector_queries=[vector_query])을 함께 사용하여 검색 합니다.
results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure Storage
Score: 0.03306011110544205
Content: Azure Storage is a scalable, durable, and highly available cloud storage service that supports a variety of data types, including blobs, files, queues, and tables. It provides a massively scalable object store for unstructured data. Storage supports data redundancy and geo-replication, ensuring high durability and availability. It offers a variety of data access and management options, including REST APIs, SDKs, and Azure Portal. You can secure your data using encryption at rest and in transit.
Category: Storage

Title: Azure Table Storage
Score: 0.032258063554763794
Content: Azure Table Storage is a fully managed, NoSQL datastore that enables you to store and query large amounts of structured, non-relational data. It provides features like automatic scaling, schema-less design, and a RESTful API. Table Storage supports various data types, such as strings, numbers, and booleans. You can use Azure Table Storage to store and manage

In [31]:
# 위의 검색 결과와, 벡터 검색만 사용(search_text=None) 의 결과를 비교해 보면, 다르다는 것을 알 수 있습니다.
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector")

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure Blob Storage
Score: 0.85977167
Content: Azure Blob Storage is a scalable, durable, and high-performance object storage service for unstructured data. It provides features like data redundancy, geo-replication, and fine-grained access control. Blob Storage supports various data types, such as images, documents, and videos. You can use Blob Storage to store and manage your data, build data lakes, and develop big data analytics solutions. It also integrates with other Azure services, such as Azure CDN, Azure Functions, and Azure Machine Learning.
Category: Storage

Title: Azure Storage
Score: 0.859018
Content: Azure Storage is a scalable, durable, and highly available cloud storage service that supports a variety of data types, including blobs, files, queues, and tables. It provides a massively scalable object store for unstructured data. Storage supports data redundancy and geo-replication, ensuring high durability and availability. It offers a variety of data access and man

### 하이브리드(hybrid) 검색에 가중치(weight) 적용

벡트 검색에 가중치(weight)를 조정 할 수 있습니다.

In [33]:
query = "scalable storage solution"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector", weight=0.2)

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure Storage
Score: 0.019945355132222176
Content: Azure Storage is a scalable, durable, and highly available cloud storage service that supports a variety of data types, including blobs, files, queues, and tables. It provides a massively scalable object store for unstructured data. Storage supports data redundancy and geo-replication, ensuring high durability and availability. It offers a variety of data access and management options, including REST APIs, SDKs, and Azure Portal. You can secure your data using encryption at rest and in transit.
Category: Storage

Title: Azure Table Storage
Score: 0.019354838877916336
Content: Azure Table Storage is a fully managed, NoSQL datastore that enables you to store and query large amounts of structured, non-relational data. It provides features like automatic scaling, schema-less design, and a RESTful API. Table Storage supports various data types, such as strings, numbers, and booleans. You can use Azure Table Storage to store and manag

### 시맨틱 하이브리드(Semantic Hybrid) 검색

- 하이브리드 검색 결과를 semantic ranker 를 사용하여 캡션 된 결과를 반환 합니다.
- vectdor index 생성 시 sementic 설정([SemanticConfiguration](https://learn.microsoft.com/ko-kr/python/api/azure-search-documents/azure.search.documents.indexes.models.semanticconfiguration))을 추가 하였으며, 시맨틱 체계 순위, 캡션, 강조 표시와 답변에 사용할 제목, 콘텐츠 및 키워드 필드를 설정하였습니다.

``` python
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="category")],
        content_fields=[SemanticField(field_name="content")]
    )
)
```



In [39]:
query = "what is azure sarch?"

vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector", exhaustive=True)

# 하이브리드 검색
results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='my-semantic-config', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Semantic Answer: Azure Cognitive Search. AI + Machine Learning. Azure Cognitive Search is<em> a fully managed search-as-a-service that enables you to build rich search experiences for your applications.</em> It provides features like full-text search, faceted navigation, and filters. Azure Cognitive Search supports various data sources, such as Azure SQL Database, Azure Blob Storage, and Azure Cosmos DB. You can use Azure Cognitive Search to index your data, create custom scoring profiles, and integrate with other Azure services. It also integrates with other Azure services, such as Azure Cognitive Services and Azure Machine Learning..
Semantic Answer Score: 0.9287109375

Title: Azure Stack Edge
Reranker Score: 2.1050379276275635
Content: Azure Stack Edge is a managed, edge computing appliance that enables you to run Azure services and AI workloads on-premises or at the edge. It provides features like hardware-accelerated machine learning, local caching, and integration with Azure IoT 